
## Introduction

With the launch of AmberLens, analysts and researchers can now visualize and derive significant insights into various facets of the digital asset world. The platform offers a range of dashboards that highlight key aspects of different domains in the space as well as topics that are top of mind for the industry. While these metrics are highly valuable for understanding macro trends, leveraging these insights for more granular, short-term trading is a non-trivial task.

In this three part series, accompanied with a Databricks notebook providing all the code used, we will elucidate how to utilize AmberLens to the fullest. Traders and analysts will be enabled to dive deeper and discover new insights as the series outlines how to create a Bitcoin trading strategy that leverages AmberLens’ technical indicators.

In Part 1 of this series, we will do some basic feature engineering to create new metrics and evaluate their usefulness. In Part 2, we will use these features to train and test machine learning models. Finally, in Part 3, we will discuss how to use these features and machine learning models to create a trading strategy.



## Reading in the Data

Here is a list of the Bitcoin metrics we have available to use: 

* Net Unrealized Profit / Loss (NUPL)
* MVRV Z-Score
* Percent of Addresses / Supply in Profit
* Reserve Risk
* Stock-To-Flow
* Puell Multiple
* Miner Supply Spent vs. Sold
* Hodl Net Position Change
* Bitcoin Yardstick
* Monthly Active Addresses
* Monthly New Addresses / Momentum
* Liquid vs. Illiquid Supply
* Address Balances Buckets (BTC and USD)
* HODL Waves
* Miner Position Index

With the launch of AmberLens, analysts and researchers can now visualize and derive significant insights into various facets of the digital asset world. The platform offers a range of dashboards that highlight key aspects of different domains in the space as well as topics that are top of mind for the industry. While these metrics are highly valuable for understanding macro trends, leveraging these insights for more granular, short-term trading is a non-trivial task.

In this three part series, accompanied with a Databricks notebook providing all the code used, we will elucidate how to utilize AmberLens to the fullest. Traders and analysts will be enabled to dive deeper and discover new insights as the series outlines how to create a Bitcoin trading strategy that leverages AmberLens’ technical indicators.

In Part 1 of this series, we will do some basic feature engineering to create new metrics and evaluate their usefulness. In Part 2, we will use these features to train and test machine learning models. Finally, in Part 3, we will discuss how to use these features and machine learning models to create a trading strategy.



In [0]:
%sql
CREATE TABLE IF NOT EXISTS <tablename>
USING snowflake
OPTIONS (
    host        '<hostname>.snowflakecomputing.com',
    port        '443',
    user        '<username>',
    password    '<password>',
    sfWarehouse '<warehouse>',
    sfRole      'DATABRICKS',
    database    'AMBERDATA_LOADING',
    schema      'RAW_MARKET',
    dbtable     'SILVER_SPOT_PRICE_BTC_USD_DAILY'
);

Then we can query these tables like normal:

In [0]:
price_df = spark.sql("select timestamp as date, price from analytics.silver_spot_price_btc_usd_daily")
nupl_df = spark.sql("select date, nupl from analytics.gold_btc_nupl_daily")
mvrv_df = spark.sql("select date, mvrvZ from analytics.gold_btc_mvrv_daily")
sip_df = spark.sql("select date, supplyInProfitPercentage from analytics.gold_btc_supply_in_profit_daily")
rr_df = spark.sql("select date, reserveRisk from analytics.gold_btc_reserve_risk_daily")
stf_df = spark.sql("select date, ratio as stockToFlow from analytics.gold_btc_stock_to_flow_daily")
pm_df = spark.sql("select date, puellMultiple from analytics.gold_btc_puell_multiple_daily")
hnpc_df = spark.sql("select date, hodlNetPositionChangeDaily as hodlNetPositionChange from analytics.gold_btc_hodl_net_position_change_daily")
by_df = spark.sql("select date, yardstick from analytics.gold_btc_yardstick_daily")
am_df = spark.sql("select date, passive_addresses, active_addresses, new_addresses, new_inputs, new_outputs, 30_day_new_address_ma, 365_day_new_address_ma from analytics.gold_btc_address_momentum_daily")
hw_df = spark.sql("""
    with max_rundate as (select max(rundate) as rundate from analytics.btc_hodl_wave)
    select 
        date, 
        utxo_value_under_1d, utxo_value_1d_1w, utxo_value_1w_1m, utxo_value_1m_3m,
        utxo_value_3m_6m, utxo_value_6m_12m, utxo_value_12m_18m, utxo_value_18m_24m, 
        utxo_value_2y_3y, utxo_value_3y_5y, utxo_value_5y_8y, utxo_value_greater_8y, 
        utxo_count_under_1d, utxo_count_1d_1w, utxo_count_1w_1m, 
        utxo_count_1m_3m, utxo_count_3m_6m, utxo_count_6m_12m, utxo_count_12m_18m, utxo_count_18m_24m, 
        utxo_count_2y_3y, utxo_count_3y_5y, utxo_count_5y_8y, utxo_count_greater_8y
    from analytics.btc_hodl_wave where rundate = (select rundate from max_rundate)""")
mpi_df = spark.sql("select date, minerPositionIndex from analytics.gold_btc_miner_position_index_daily")

For liquid/illiquid supply and address buckets, we will need to do some additional work to read them in as features. Because they are pivoted tables, we will need to "unpivot" them in order to get them in the same shape as the other datasets. 

In [0]:
from pyspark.sql.functions import col, max

lis_df = (spark.sql("select date, liqudity_rank, total_balance from analytics.gold_btc_liquid_illiquid_supply")
    .groupBy("date").pivot("liqudity_rank").agg(max("total_balance"))
    )
display(lis_df)

And join them into one df

In [0]:
df = (price_df
      .join(nupl_df, on=["date"])
      .join(mvrv_df, on=["date"])
      .join(sip_df, on=["date"])
      .join(rr_df, on=["date"])
      .join(stf_df, on=["date"])
      .join(pm_df, on=["date"])
      .join(hnpc_df, on=["date"])
      .join(by_df, on=["date"])
      .join(am_df, on=["date"])
      .join(lis_df, on=["date"])
      .join(mpi_df, on=["date"])
      .filter("date > '2015-01-01'")
)



## Exploratory Data Analysis

Now that we have the data, let's explore how we can visualize this data and draw insights from it. In Databricks, the `display` function provides very functional initial EDA tools. After we `display(df)` we can then use the data profiler to see basic summary statistics for each column, as well as the overall distribution of each column.

In [0]:
display(df)


Data profiles are a great way to get a first look at the data. Of course, since this data is from AmberLens, it is relatively clean and complete. We can see that the minimum date is 2015 and the maximum date is April 29, 2024 (this snapshot was taken on April 30, 2024). Data skew is present in several indicators, both left and right depending on the metric. Data skew implies we may need some sort of normalization technique in order to use them correctly.

## Feature engineering

### Moving Averages

Moving averages are a fundamental technique in feature engineering, particularly valuable for time series data. They serve to smooth out short-term fluctuations and highlight longer-term trends or cycles in the data, making it easier to identify underlying patterns. This smoothing effect is crucial because raw data can often be noisy, which can obscure the true signal that models need to learn from. By averaging data points over a specified period, moving averages reduce the impact of random variability, leading to more robust features for predictive modeling.

The intuition behind using moving averages lies in their ability to provide a clearer view of the data's direction over time. For instance, in financial markets, moving averages can help distinguish between random price movements and significant trends, aiding in better forecasting and decision-making. They effectively filter out the 'noise,' allowing the model to focus on the 'signal,' which is the true underlying trend. This makes moving averages particularly useful for applications like stock price prediction, weather forecasting, and demand planning, where understanding the trend over time is crucial.

For example, let's take a look at supply in profit. Suppose we have a belief that the trend or direction of supply in profit is more important than the percent of supply in profit on a given day. Particularly, we believe that the overall trend of the graph gives us signals as to whether we have hit the top or bottom of a market cycle. To smooth short term fluctuations, let's create a 7 day, 30 day, and 365 day moving average for this time series, and then plot it with price to observe the relationship between the two variables.


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import avg

w = Window().orderBy("date")

display(df
    .select("date", "price", "supplyInProfitPercentage")
    .withColumn("SIP_MA7Days", avg("supplyInProfitPercentage").over(w.rowsBetween(-7, 0)))
    .withColumn("SIP_MA30Days", avg("supplyInProfitPercentage").over(w.rowsBetween(-30, 0)))
    .withColumn("SIP_MA365Days", avg("supplyInProfitPercentage").over(w.rowsBetween(-365, 0)))

  )

Of course, there is a relationship between supply in profit and price: as price goes up, supply in profit will increase, and vice versa but what we are interested in is how supply in profit conveys market tops or bottoms.

Raw supply in profit is very noisy; by smoothing out the curve we can notice some interesting trends. For example, in the 30-day MA for supply in profit, when it clears higher than 80%, it is almost always a maximum for the curve. This is a great top indicator as it's a clear signal for participants to claim their gains before price correction. This logic also applies as a market bottom indicator: the 30-day smoothed supply in profit percentage rarely drops below 30%. This signal can act as a clear buy signal with the market likely bottoming out; we will likely see prices start to rise again. Using a simple moving average transformation, we were able to create a new metric with clear entry and exit strategies. In Part 2, we will explore how we can devise a trading strategy using these types of signals and compare them against other baseline models, and in Part 3,  we’ll explore how we can use Machine Learning to assist in feature engineering.

Continuing this example, the 7-day and 365-day smoothed supply in profit curves are less useful, since they do not provide as much information. For example, in the 7-day smoothed curve, we see it cross the 80% and 30% threshold multiple times in one market cycle, indicating that it is only slightly less noisy than the current supply in profit. The 365-day smoothed curve is too smooth, and we would only make four trades using the previous 30-day heuristic we came up with. This is not necessarily a bad thing: we can use the 365-day curve as a measure of confidence that we are in a true market top or bottom. That is, if the 365-day curve reaches either extreme we have more confidence in the market truly bottoming or topping out. This shows a few ways in which we can leverage different moving average windows to capture directionality and trends.


### Daily / Monthly Net Change

The periodic net change in a time series, defined as the difference between a set period of data points, is a powerful feature for capturing short-term dynamics and volatility within the data. The most common expression of this metric is at a daily or monthly granularity, aptly named daily net change or monthly net change. This feature engineering technique gives us insight into the immediate momentum and direction of change in a time series, providing insights into how the value of a variable evolves periodically. This exact concept is one of our metrics, HODL Net Position Change. The base time series, HODLed coins, is a useful metric by itself, but by taking the daily or monthly net change, the engineered metric gives us better insight into the magnitude at which Bitcoin users are HODLing or not HODLing their coins.

By incorporating a periodic net change as a feature, models can gain a nuanced understanding of temporal dependencies and short-term fluctuations that might be critical for accurate predictions. This feature helps in identifying patterns such as trends, reversals, and volatility spikes, which might not be evident from the raw data alone. Additionally, it can enhance the model's ability to detect anomalies, such as sudden drops or surges, which are important for risk management and decision-making processes. Overall, the periodic net change enriches the feature set, enabling more responsive and insightful modeling of time series data.

Let's look at the periodic net change for another one of our series. Suppose we have a theory that the daily change in a HODL wave gives some insight into whether the price is dropping or rising. The idea here is that as long-term bands gain or lose coins, they convey information about the state of the network (if fewer people are HODLing, then we will see a downward price pressure, and vice versa). Let's first group different waves into three different bands to make analysis easier: short-term HODLers (< 3 months), mid-term HODLers (3 months - 3 years) and long-term HODLers (1 year+).



In [0]:
display(hw_df.orderBy("date"))

In [0]:
from pyspark.sql.functions import col

behavior_df = (hw_df
    .withColumn("short_term_hodlers", col("utxo_value_under_1d") + col("utxo_value_1d_1w") + col("utxo_value_1w_1m") + col("utxo_value_1m_3m"))
    .withColumn("mid_term_hodlers", col("utxo_value_3m_6m") + col("utxo_value_6m_12m") + col("utxo_value_12m_18m") + col("utxo_value_18m_24m"))
    .withColumn("long_term_hodlers", col("utxo_value_2y_3y") + col("utxo_value_3y_5y") + col("utxo_value_5y_8y") + col("utxo_value_greater_8y"))
    .select("date", "short_term_hodlers", "mid_term_hodlers", "long_term_hodlers") 
    .join(price_df, on=["date"])       
)

display(behavior_df.orderBy(col("date").desc()))

Some initial observations after grouping: short-term holders experience a lot more variance in their wave value, while mid to long-term HODLers experience relatively longer sinusoidal waves with lower variation in their magnitude. This might lead us to believe that short-term HODL waves can give us some insight into price or price volatility. Volatility is typically measured as the standard deviation of log returns multiplied by the square root of periods (in this case, 365 since crypto trades year-round). Coincidentally, to model volatility we will also need to take the daily difference of price to get daily return, which showcases another use case for daily net change. We choose to look at price volatility because some trade strategies or machine learning models want to model volatility as entry or exit signals, and so building a feature to model volatility can be very useful.

Volatility is typically measured as the standard deviation of log returns multiplied by the square root of periods (in this case, 365 days). To model volatility we will also need to take the daily difference of price to get daily return, which showcases another use case for daily net change. Let's model to see if we can capture this relationship.



In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, log, stddev, sqrt, lit

w = Window.orderBy("date")

display(
    behavior_df
    .withColumn("short_monthly_net_position_change", col("short_term_hodlers") - lag("short_term_hodlers", 30).over(w))
    .withColumn("mid_monthly_net_position_change", col("mid_term_hodlers") - lag("mid_term_hodlers", 30).over(w))
    .withColumn("long_monthly_net_position_change", col("long_term_hodlers") - lag("long_term_hodlers", 30).over(w))
    .withColumn("price_returns_log", log(col("price") / lag("price", 1).over(w)))
    .withColumn("price_volatility_rolling", stddev("price_returns_log").over(Window().orderBy("date").rowsBetween(-30, 0)) * sqrt(lit(365) / lit(30))) 
)


Let's break down each step of code here:

```
    .withColumn("short_monthly_net_position_change", col("short_term_hodlers") - lag("short_term_hodlers", 30).over(w))
    .withColumn("mid_monthly_net_position_change", col("mid_term_hodlers") - lag("mid_term_hodlers", 30).over(w))
    .withColumn("long_monthly_net_position_change", col("long_term_hodlers") - lag("long_term_hodlers", 30).over(w))
```

Since HODL waves are updated monthly, we want to look at monthly net position change. This line gives us a rough approximation of monthly net position change. Months vary in length from 29 - 31 days, and we're more interested in general trends than being exact with our monthly calculation. When productionalizing this code, we can be more rigorous with ensuring we are looking at the exact net difference at the start of each month, but an approximate version will work well enough for now, since we're still doing exploratory data analysis. These three lines are simply taking each of our aggregated HODL waves, and then subtracting from the value 30 days ago. 

The next two lines show how to calculate price volatility

```
    .withColumn("price_returns_log", log(col("price") / lag("price", 1).over(w)))
    .withColumn("price_volatility_rolling", stddev("price_returns_log").over(Window().orderBy("date").rowsBetween(-30, 0)) * sqrt(lit(365) / lit(30))) 
```

Because Bitcoin is such a volatile stock, we want to look at the 30 day rolling volaility, because Bitcoin goes through short market cycles with strong volatility changes. Volatility is typically measured via log returns, and log returns is the log(price - prev_price). Using log laws, we can simplify this to log(price / prev_price), which is what the first line of code is showing. Then, we take a 30-day rolling standard deviation of log returns, and multiply by the annualization factor. Typically, this should be `sqrt(# of trading days) / (rolling window))` where the numerator is usually 252. However, since Bitcoin trades every day, we have an annualization factor of 365 / 30. 





Now that we have our net position change and price volatility, we can look at the relationship between the two variables. In the short-term wave, we see a very slight positive relationship between the monthly wave position change and price volatility. This confirms our initial hypothesis, which was that as the short-term wave gets larger, price volatility increases. The mid-term wave has a very flat, almost no relationship, and the long-term wave has a relatively strong negative correlation. This is fascinating because it shows that there is another relationship we didn’t account for the negative correlation between long-term waves position change and price volatility. This implies that as more people HODL for longer periods, price volatility decreases as well, which is logically sound.  This means we now have two features we can use as features to predict price volatility, as any correlation can be utilized as feature inputs into a price volatility model.


### Conclusion

Feature engineering is the first step in machine learning, as we need strong features to have strong predictive models. By showcasing two commonly used feature engineering techniques, we hope you can feel inspired to investigate similar theories or hypotheses using AmberLens data. In the next part, we will go over how we can use some of these features to build basic trading strategies that leverage these metrics for trading.